In [ ]:
#reconstruction script draft
# General imports
import sys
import math
import copy
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import json
import os


sys.path.insert(0,'./neuro_dot')
from datetime import datetime
from scipy.io import savemat
from Visualizations import viz
from Spatial_Transforms import sx4m
from Temporal_Transforms import tx4m
from Light_Modeling import lmdl
from File_IO import io
from Analysis import anlys
from Reconstruction import recon
from Matlab_Equivalent_Functions import matlab

In [ ]:
# To get started, edit this cell for folder paths; EDIT AT THE BEGINNING ONLY
## For papermill - this is the tagged parameters cell 
saveImages = 'yes' # Options are yes/no to save output figures (must be lowercase 'yes' or 'no')
saveMatPath ="./Output" # Path to folder to save output, figures will be saved to subdirectory called 'Output_Images'
saveNoteBookPath = "./Output" # Path to folder to save output, Jupyter Notebook will be saved to subdirectory called 'Output_Notebook'
savedMatricies = {}
# TO MAKE CHANGES TO FIGURES, EDIT ONLY THIS CELL
# patient_data = "/input/subjectdata.mat"  # Path to data; an example data set has been provided here
participant_data = "./Subject/NeuroDOT_Data_Sample_CCW1.mat"
params_file = "./Project/params_files/params.txt"
Emat = './Project/E_matricies/E.mat'
MNI_file = './Project/MNI_files/Segmented_MNI152nl_on_MNI111_py.mat'
A_fn=""
A = None
E = None 
MNI = None 

In [ ]:
if saveImages == 'no':
    print('SAVED IMAGES IS SET TO NO. NO IMAGES WILL BE SAVED.')
    
z = ""
with open(params_file) as f:
    for line in f: 
        z = z + line.strip()
params = json.loads(z)
# If you want to change an individual or a few params, just follow syntax below (NOTE: make changes before 'print' statements to confirm dictionary updates): 
# params['bthresh'] = 0.075
print("Params being used: \n")
print(params)
A_size = os.path.getsize(A_fn)/1000
print("A size", A_size)

In [ ]:
# Load your data file, or one of the provided data samples using the absolute path to the file (.mat format)
data = io.loadmat(participant_data)['data'] 
info = io.loadmat(participant_data)['info'] 
tag = participant_data.split("/")[-1].split("_")[-1][:-4]
# flags = io.loadmat(participant_data)['flags']
# params = {}                                 # create empty params dictionary to hold dt and tp for block averaging WHERE RESET OCCURSSS 

In [ ]:
# Set parameters for A and block length for quick processing examples
#NOTE: THIS CODE IS SPECIFIC TO LANGUAGE DATA SET AND NEURODOT SAMPLE DATA
# if 'CCW1' in participant_data or 'CCW2' in participant_data or 'CW1' in participant_data or 'OUT' in participant_data:
#     A_fn='/Project/A_matricies/A_AdultV24x28.mat' #  Sensitivity Matrix (path to A matrix on your machine! this file is too large to go in the GitHub repo being worked out of)
#     params['dt']=36                     # Block length
#     params['tp']=16                     # Example (block averaged) time point
# if 'IN1' in participant_data:
#     A_fn='/Project/A_matricies/A_AdultV24x28.mat' # Sensitivity Matrix
#     params['dt']=36                     # Block length
#     params['tp']=32                     # Example (block averaged) time point

# if  'HW1'in participant_data or'HW2' in participant_data or'RW1' in participant_data or'GV1'in participant_data or'HW3-Noisy' in participant_data:
#     A_fn='/Project/A_matricies/A_Adult_96x92.mat'  # Sensitivity Matrix
#     params['dt']=30                     # Block length
#     params['tp']=16                     # Example (block averaged) time point

In [ ]:
## PRE-PREOCESSING PIPELINE
lmdata = tx4m.logmean(data)[0]                                    # Logmean Light Levels
info = anlys.FindGoodMeas(lmdata, info, 0.075)                    # Detect Noisy Channels
gvtd = anlys.CalcGVTD(lmdata)
info['GVTD'] = gvtd
viz.nlrGrayPlots_220324(lmdata,info)                              # Gray Plot with synch points
lmdata = tx4m.detrend_tts(lmdata)                                 # Detrend Data
hpdata = tx4m.highpass(lmdata, .02, info['system']['framerate'])  # High Pass Filter (0.02 Hz)
lp1data = tx4m.lowpass(hpdata, 1, info['system']['framerate'])    # Low Pass Filter 1 (1.0 Hz)
hem = tx4m.gethem(lp1data, info)                                  # Superficial Signal Regression
lp1data, _ = tx4m.regcorr(lp1data, info, hem)
lp2data = tx4m.lowpass(lp1data, 0.2, info['system']['framerate']) # Low Pass Filter 2 (0.5 Hz)
resample_data, info = tx4m.resample_tts(lp2data, info, 1, 1e-5)   # 1 Hz Resampling (1 Hz)

In [ ]:
## View pre-processed data
keep = np.logical_and(np.logical_and(np.where(info['pairs']['WL'] == 2,1,0), np.where(info['pairs']['r2d'] < 40,1,0)), info['MEAS']['GI']) # measurements to include

fig = plt.figure(dpi = 150, tight_layout = True)
gs = gridspec.GridSpec(3,1)
ax1 =  plt.subplot(gs[0,0])
ax2 =  plt.subplot(gs[1,0])
ax3 =  plt.subplot(gs[2,0])

ax1.plot(np.transpose(resample_data[keep,:]), linewidth = 0.2) # plot signals 
ax1.set_xlabel('Time (samples)')
ax1.set_ylabel('log(\u03A6/$\u03A6_{0}$)')  

im2 = ax2.imshow(resample_data[keep,:], aspect = 'auto')
ax2.set_xlabel('Time (samples)')
ax2.set_ylabel('Measurement #') 
xplot = np.transpose(np.reshape(np.mean(np.transpose(resample_data[keep,:]),1), (len(np.mean(np.transpose(resample_data[keep,:]),1)),1)))

ftdomain, ftmag,_,_ = tx4m.fft_tts(xplot,info['system']['framerate']) # Generate average spectrum
ftmag = np.reshape(ftmag, (len(ftdomain)))
ax3.semilogx(ftdomain,ftmag, linewidth = 0.5) # plot vs. log frequency
ax3.set_xlabel('Frequency (Hz)')
ax3.set_ylabel('|X(f)|') 
ax3.set_xlim([1e-3, 1])

viz.nlrGrayPlots_220324(resample_data,info)  # Gray Plot with synch points

In [ ]:
## Block Averaging the measurement data and view
badata = anlys.BlockAverage(resample_data, info['paradigm']['synchpts'][info['paradigm']['Pulse_2']], params['dt'])[0]

mean_badata = np.ndarray.mean(badata, axis = 1,dtype = np.float64)
badata = badata - mean_badata[:,None]

keep = np.logical_and(np.logical_and(np.where(info['pairs']['WL'] == 2,1,0), np.where(info['pairs']['r2d'] < 40,1,0)), info['MEAS']['GI']) 

fig = plt.figure(dpi = 150, tight_layout = True)
gs = gridspec.GridSpec(2,1)
ax1 =  plt.subplot(gs[0,0])
ax2 =  plt.subplot(gs[1,0])

ax1.plot(np.transpose(badata[keep,:]), linewidth = 0.15) # plot signals 
ax1.set_xlabel('Time (samples)')
ax1.set_ylabel('log(\u03A6/$\u03A6_{0}$)')  

#the following two lines may be needed if there is any issue with the scaling of the y axis in the image view
# m=np.amax(np.amax(abs(badata[keep,:])))
# image_y = np.arange(-1,3,2)*m
im2 = ax2.imshow(badata[keep,:], aspect = 'auto')
ax2.set_xlabel('Time (samples)')
ax2.set_ylabel('Measurement #') 

In [ ]:
A = None
if A is None:
    A = {}
    if  A_size > 2000000:
        A['A'] = io.loadmat7p3(A_fn)['A']
        A['infoA'] = io.loadmat7p3(A_fn)['info']
        for key in A['infoA']['tissue']['dim']:
            if type(A['infoA']['tissue']['dim'][key]) == int or type(A['infoA']['tissue']['dim'][key]) == float :
                if np.size(A['infoA']['tissue']['dim'][key]) ==1:
                    A['infoA']['tissue']['dim'][key] = int(A['infoA']['tissue']['dim'][key])
    else:
        print("ELSE")
        A['A'] = io.loadmat(A_fn)['A']
        A['infoA'] = io.loadmat(A_fn)['info']

    if len(A['A'].shape)>2:  # A data structure [wl X meas X vox]-->[meas X vox] # refer to matlab code for this
        Nwl = A['A'].shape[0]
        Nmeas = A['A'].shape[1]
        Nvox = A['A'].shape[2]
        A['A']=np.reshape(np.transpose(A['A'],(1,0,2)),Nwl*Nmeas,Nvox) # Changed permute to transpose 

Nvox= A['A'].shape[1]
Nt=resample_data.shape[1]
cortex_mu_a=np.zeros((Nvox,Nt,2))

for j in range(1,3): 
    keep = np.logical_and(np.logical_and(np.where(info['pairs']['WL'] == j,1,0), np.where(info['pairs']['r2d'] <= 40,1,0)), info['MEAS']['GI']) 
    print('> Inverting A')                
    iA = recon.Tikhonov_invert_Amat(A['A'][keep, :], 0.01, 0.1)                 # Invert A-Matrix
    print('> Smoothing iA')
    iA = recon.smooth_Amat(iA, A['infoA']['tissue']['dim'], 2)                  # Smooth Inverted A-Matrix      
    cortex_mu_a[:, :, j-1] = recon.reconstruct_img(resample_data[keep, :], iA)  # Reconstruct Image Volume 
savedMatricies["cortex_mu_a"] = copy.deepcopy(cortex_mu_a)

# Spectroscopy
if E is None:    
    E = io.loadmat(Emat)['E']
cortex_Hb = recon.spectroscopy_img(cortex_mu_a, E) 
savedMatricies["cortex_Hb"] = copy.deepcopy(cortex_Hb)

cortex_HbO = cortex_Hb[:, :, 0]
cortex_HbR = cortex_Hb[:, :, 1]
cortex_HbT = cortex_HbO + cortex_HbR

In [ ]:
## Select Volumetric visualizations of block averaged data
if (MNI is None):
    MNI=io.loadmat(MNI_file)['vol'] # load MRI (same data set as in A matrix dim)
    infoB=io.loadmat(MNI_file)['h']
MNI_dim = sx4m.affine3d_img(MNI,infoB,A['infoA']['tissue']['dim'],affine =np.eye(4),interp_type ='nearest') # transform to DOT volume space  ################################## save ######################################
savedMatricies["MNI_dim"] = copy.deepcopy(MNI_dim)

In [ ]:
# Flat Field Reconstruction
a=np.squeeze(A['A'][keep,:])

ffr = lmdl.makeFlatFieldRecon(a, iA)
fooV = sx4m.GoodVox2vol(ffr, A['infoA']['tissue']['dim'])
fooV = fooV/np.amax(fooV)

Params_recon = {}
Params_recon['Th'] = {}
Params_recon['Scale']=1
Params_recon['Th']['P']=1e-2
Params_recon['Th']['N']=-Params_recon['Th']['P']
Params_recon['Cmap']='jet' ################################## maybe save these params - they are input solely to PSTT for visualization ######################################
Params_recon['slices'] = [34,5,12]

viz.PlotSlices(MNI_dim, infoB, params = Params_recon, overlay = fooV) # set coordinates manually and save image in XNAT container (Maybe just do %matplotlib inline)
plt.close() #need to find someway to kill interactive process in container so that it will run to completion and save out .png of the ffr

In [ ]:
## Explore the block-averaged data interactively
badata_HbO = anlys.BlockAverage(cortex_HbO, info['paradigm']['synchpts'][info['paradigm']['Pulse_2']-1], params['dt'])[0]

first_col = badata_HbO[:,0]
badata_HbO= badata_HbO - first_col[:,None]

badata_HbOvol = sx4m.GoodVox2vol(badata_HbO,A['infoA']['tissue']['dim'])

Params_recon = {}
Params_recon['Th'] = {}
Params_recon['Scale']=0.8*np.amax(abs(badata_HbOvol))
Params_recon['Th']['P']=0.2*Params_recon['Scale']
Params_recon['Th']['N']=-Params_recon['Th']['P']
Params_recon['Cmap']='jet' ################################## maybe save these params - they are input solely to PSTT for visualization ######################################
savedMatricies["Params_recon"] = copy.deepcopy(Params_recon)

#Uncomment the lines below if you want to see the interactive graph. 
# %matplotlib
# plt.ion()

# viz.PlotSlicesTimeTrace(MNI_dim,A['infoA']['tissue']['dim'],Params_recon,badata_HbOvol,info)
# plt.close()


In [ ]:
## Visualize block-averaged absorption and HbR, HbT data 
# use either of the next two lines to select mu_a (wavelength of 1 or 2 in the third
# dimension) or HbO, HbR, or HbT (second line)
# badata_HbO = anlys.BlockAverage(cortex_mu_a[:,:,1], info['paradigm']['synchpts'][info['paradigm']['Pulse_2]], info['dt'])
badata_HbT = anlys.BlockAverage(cortex_HbT, info['paradigm']['synchpts'][info['paradigm']['Pulse_2']-1], params['dt'])[0]
first_col = badata_HbT[:,0]
badata_HbT= badata_HbT - first_col[:,None]
badata_HbTvol = sx4m.GoodVox2vol(badata_HbT,A['infoA']['tissue']['dim'])

Params_recon = {}
Params_recon['Th'] = {}
Params_recon['Scale']=0.8*np.amax(abs(badata_HbTvol)) ##Modifiable
Params_recon['Th']['P']=0.2*Params_recon['Scale'] ##Modifiable
Params_recon['Th']['N']=-Params_recon['Th']['P'] ##Modifiable
Params_recon['Cmap']='jet'

#NOTE: visulization is turned off for the container (line below) if you would like to see this after downloading from XNAT uncomment this line and rerun with appropriate data 
# Params_recon['slices'] = [49, 8, 17] # optional: choose which slices to view in PlotSlicesTimeTrace
# viz.PlotSlicesTimeTrace(MNI_dim,A['infoA']['tissue']['dim'],Params_recon,badata_HbOvol,info)

In [ ]:
participant_used = participant_data.split('/')[-1][:-4]
params_used = params_file.split('/')[-1][:-4]
file_name = participant_used+"_"+params_used+".mat"
savePath = saveMatPath+"/"+file_name
print("saving file to: " + savePath)


#Save Matricies generated name format: participant+paramsused+datatime
savemat(savePath, savedMatricies)